In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

<IPython.core.display.Javascript object>

In [3]:
import warnings

warnings.filterwarnings("ignore")

from dask.distributed import Client, progress

client = Client(n_workers=6, threads_per_worker=1, memory_limit="2GB")
client

Client Scheduler: tcp://127.0.0.1:52988 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 6 Memory: 12.00 GB


<IPython.core.display.Javascript object>

In [4]:
df = dd.read_csv("data/creditcard.csv", dtype={"Time": "float64"})

<IPython.core.display.Javascript object>

In [5]:
from dask_ml.model_selection import train_test_split

<IPython.core.display.Javascript object>

In [7]:
X = df.drop("Class", 1)
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.persist()
X_test.persist()
y_train.persist()
y_test.persist()

Dask Series Structure:
npartitions=3
    int64
      ...
      ...
      ...
Name: Class, dtype: int64
Dask Name: split, 3 tasks

<IPython.core.display.Javascript object>

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import roc_auc_score
import joblib

<IPython.core.display.Javascript object>

In [11]:
rfc = RandomForestClassifier()

with joblib.parallel_backend("dask"):
    scores = cross_validate(rfc, X_train.compute(), y_train.compute(), cv=4)
scores

{'fit_time': array([108.97011089, 111.68510866, 106.83010697, 109.77510786]),
 'score_time': array([0.24199843, 0.23800516, 0.39400005, 0.24800158]),
 'test_score': array([0.02000281, 0.99945463, 0.99950741, 0.99940185])}

<IPython.core.display.Javascript object>

In [13]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

with joblib.parallel_backend("dask"):
    scores = cross_validate(lr, X_train.compute(), y_train.compute(), cv=4)
scores

{'fit_time': array([5.01299739, 5.09104085, 5.11802649, 4.98500085]),
 'score_time': array([0.0140059 , 0.01098776, 0.01400137, 0.02899933]),
 'test_score': array([0.98710461, 0.99912037, 0.99941944, 0.99822314])}

<IPython.core.display.Javascript object>

In [14]:
from sklearn.svm import SVC

svc = SVC()

with joblib.parallel_backend("dask"):
    scores = cross_validate(svc, X_train.compute(), y_train.compute(), cv=4)
scores

{'fit_time': array([184.66301799,   9.06599879,   9.14199924,   8.89399862]),
 'score_time': array([1.40199995, 1.40202904, 1.34200358, 1.38000059]),
 'test_score': array([0.99827592, 0.99827592, 0.99825833, 0.99825833])}

<IPython.core.display.Javascript object>

Looking purely at the fit time Logistic Regression fit the quickest which is to be expected. The Random Forest takes the most amount of time. The SVC is the most confusing. Only one of the fit times is long. I wonder if something weird is going on with the train test / manifold split. They all score surprisingly high which is almost suspicious. Considering everything, my first choice would certainly be logistic regression. Better to go simple especially with such high accuracy scores.

In [ ]:
client.close()